In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
start_date="2018-01-01"
end_date="2023-12-31"
tickers=["AAPL","^GSPC","MSFT"]
data1=yf.download('AAPL',start=start_date,end=end_date)
data2=yf.download('^GSPC',start=start_date,end=end_date)
data3=yf.download('MSFT',start=start_date,end=end_date)

#for i in range(len(tickers)):
#    df=pd.concat([df,yf.download(tickers[i],start=start_date,end=end_date)],ignore_index=True)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


In [37]:
data1.rename(columns={'Open':'AAPL_Open'},inplace=True)
data1.rename(columns={'High':'AAPL_High'},inplace=True)
data1.rename(columns={'Low':'AAPL_Low'},inplace=True)
data1.rename(columns={'Close':'AAPL_Close'},inplace=True)
data1.rename(columns={'Adj Close':'AAPL_Adj Close'},inplace=True)
data1.rename(columns={'Volume':'AAPL_Volume'},inplace=True)

data2.rename(columns={'Open':'^GSPC_Open'},inplace=True)
data2.rename(columns={'High':'^GSPC_High'},inplace=True)
data2.rename(columns={'Low':'^GSPC_Low'},inplace=True)
data2.rename(columns={'Close':'^GSPC_Close'},inplace=True)
data2.rename(columns={'Adj Close':'^GSPC_Adj Close'},inplace=True)
data2.rename(columns={'Volume':'^GSPC_Volume'},inplace=True)

data3.rename(columns={'Open':'MSFT_Open'},inplace=True)
data3.rename(columns={'High':'MSFT_High'},inplace=True)
data3.rename(columns={'Low':'MSFT_Low'},inplace=True)
data3.rename(columns={'Close':'MSFT_Close'},inplace=True)
data3.rename(columns={'Adj Close':'MSFT_Adj Close'},inplace=True)
data3.rename(columns={'Volume':'MSFT_Volume'},inplace=True)


In [38]:
data=pd.merge(data1,data2,on='Date',how='right')
data=pd.merge(data,data3,on='Date',how='right')
data.to_csv('AAPL,MSFT,SPX.csv')
data_corection=torch.as_tensor(data.values,dtype=torch.float32)
data_corection=data_corection.to(device)

data.to_csv('short_stocks.csv')

data_row=data_corection.t()
data_row.to(device)
data_row.size()
data_corection.size()
data 

,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Adj Close,AAPL_Volume,^GSPC_Open,^GSPC_High,^GSPC_Low,^GSPC_Close,^GSPC_Adj Close,^GSPC_Volume,MSFT_Open,MSFT_High,MSFT_Low,MSFT_Close,MSFT_Adj Close,MSFT_Volume
Date,,,,,,,,,,,,,,,,,,
2018-01-02,42.540001,43.075001,42.314999,43.064999,40.670971,102223600,2683.729980,2695.889893,2682.360107,2695.810059,2695.810059,3397430000,86.129997,86.309998,85.500000,85.949997,80.080925,22483800
2018-01-03,43.132500,43.637501,42.990002,43.057499,40.663898,118071600,2697.850098,2714.370117,2697.770020,2713.060059,2713.060059,3544030000,86.059998,86.510002,85.970001,86.349998,80.453606,26061400
2018-01-04,43.134998,43.367500,43.020000,43.257500,40.852772,89738400,2719.310059,2729.290039,2719.070068,2723.989990,2723.989990,3697340000,86.589996,87.660004,86.570000,87.110001,81.161728,21912000
2018-01-05,43.360001,43.842499,43.262501,43.750000,41.317898,94640000,2731.330078,2743.449951,2727.919922,2743.149902,2743.149902,3239280000,87.660004,88.410004,87.430000,88.190002,82.167953,23407100
2018-01-08,43.587502,43.902500,43.482498,43.587502,41.164429,82271200,2742.669922,2748.510010,2737.600098,2747.709961,2747.709961,3246160000,88.199997,88.580002,87.599998,88.279999,82.251816,22113000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,195.179993,195.410004,192.970001,193.600006,193.353287,37122800,4753.919922,4772.939941,4736.770020,4754.629883,4754.629883,3046770000,373.679993,375.179993,372.709991,374.579987,373.888580,17091100
2023-12-26,193.610001,193.889999,192.830002,193.050003,192.803986,28919300,4758.859863,4784.720215,4758.450195,4774.750000,4774.750000,2513910000,375.000000,376.940002,373.500000,374.660004,373.968445,12673100
2023-12-27,192.490005,193.500000,191.089996,193.149994,192.903839,48087700,4773.450195,4785.390137,4768.899902,4781.580078,4781.580078,2748450000,373.690002,375.059998,372.809998,374.070007,373.379547,14905400


In [39]:
class AI(nn.Module):
    def __init__(self,nUnits,nLayers):
        super(AI,self).__init__()
        
        # create dictionary to store the layers
        self.layers = nn.ModuleDict()
        self.nLayers = nLayers

        ### input layer
        self.layers['input'] = nn.Linear(18,1509)
        self.layers['hidden0']=nn.Linear(1509,nUnits)
        
        ### hidden layers
        for i in range(nLayers):
            self.layers[f'hidden{i+1}'] = nn.Linear(nUnits,nUnits)

        ### output layer
        self.layers['output'] = nn.Linear(nUnits,1)
    def forward(self,x):
        x=F.relu(self.layers['input'](x))
        for i in range(self.nLayers):
            x=F.relu(self.layers[f'hidden{i}'](x))
        x=self.layers['output'](x)
        return x
        


In [40]:
nUnitsPerLayer=512
nLayers=100
net=AI(1509,nLayers)
net=net.to(device)


In [41]:
target=data['AAPL_High']
target=target[1:]
target=torch.tensor(target.values)
target.to(device)

tensor([ 43.6375,  43.3675,  43.8425,  ..., 193.5000, 194.6600, 194.4000],
       dtype=torch.float64)

In [42]:
def trainTheModel(theModel,numepochs):
    lossfun=nn.SmoothL1Loss()
    optimizer=torch.optim.RMSprop(theModel.parameters(),lr=.01,momentum=.9)
    losses=torch.zeros(numepochs).to(device)
    for epochi in range(numepochs):
        print(epochi)
        yHat=theModel(data_row)
        loss=lossfun(yHat,target)
        losses[epochi]=loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return losses
    

In [44]:
numepochs=10000
losses=trainTheModel(net,numepochs)
plt.plot(losses.detach(),'o',markerfacecolor='w',linewidth=.1)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

0


RuntimeError: mat1 and mat2 shapes cannot be multiplied (18x1509 and 18x1509)